In [10]:
import numpy as np
import pandas as pd
import re


In [11]:
titanic = pd.read_csv("Datasets/Titanic-Dataset.csv")
print(titanic.info())
print(titanic.describe())
titanic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
titanic = titanic.drop(columns=['PassengerId', 'Cabin'])

In [13]:
titanic['Age'].fillna(titanic['Age'].mean(), inplace=True)
titanic = titanic[titanic['Embarked'].notna()]
titanic.reset_index()

/tmp/ipykernel_3214245/1621599864.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['Age'].fillna(titanic['Age'].mean(), inplace=True)


,index,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
1,1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
2,2,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
3,3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
4,4,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
884,886,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,S
885,887,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,S
886,888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,S
887,889,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C


In [14]:
titanic['Sex'] = titanic['Sex'].map({'male': 1, 'female': 0})

In [15]:
titanic['Title']=titanic['Name'].str.split(", ",expand=True)[1].str.split(".",expand=True)[0]
titanic['Title'] = titanic['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 0)
titanic['Title'] = titanic['Title'].replace(['Ms','Mlle', 'Miss'], 1)
titanic['Title'] = titanic['Title'].replace(['Mme', 'Mrs'], 2)
titanic['Title'] = titanic['Title'].replace(['Mr'], 3)
titanic['Title'] = titanic['Title'].replace(['Master'], 4)

/tmp/ipykernel_3214245/3198012624.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  titanic['Title'] = titanic['Title'].replace(['Master'], 4)


In [16]:
titanic['Family_size']=titanic['SibSp'] + titanic['Parch'] + 1
titanic.drop(columns=['Name','Parch','SibSp'],inplace=True)

In [17]:
def split_ticket_parts(s):
    if pd.isna(s):
        return (None, None)

    s = str(s).strip()

    # find the numeric part (last group of digits in the string)
    match = re.search(r'(\d+)\s*$', s)
    if not match:
        return (s, None)

    number = match.group(1)
    prefix = s[:match.start()].rstrip(' ,.')
    if prefix:
        prefix = re.sub(r'[/. ]', '', prefix)
    else:
        prefix = 'Unknown'
    return (prefix, int(number))


In [18]:
titanic[['Ticket_prefix', 'Ticket_number']] = titanic['Ticket'].apply(
    lambda s: pd.Series(split_ticket_parts(s))
)

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
titanic['Ticket_prefix_code'] = le.fit_transform(titanic['Ticket_prefix'])
titanic['Embarked_code'] = le.fit_transform(titanic['Embarked'])
titanic=titanic.drop(columns=['Ticket', 'Ticket_prefix', 'Embarked'])

In [20]:
titanic.dropna(inplace=True)

X = titanic.drop(columns='Survived')
y = titanic['Survived']

X.to_csv("preprocessed_datasets/titanic_data.csv", index=False)
y.to_csv("preprocessed_datasets/titanic_target.csv", index=False)

In [ ]:
#X = pd.read_csv("preprocessed_datasets/titanic_data.csv")
#y = pd.read_csv("preprocessed_datasets/titanic_target.csv").squeeze()

In [1]:
import pandas as pd
y = pd.read_csv("preprocessed_datasets/titanic_target.csv").squeeze()

y.value_counts(normalize=True) * 100



Survived
0    61.694915
1    38.305085
Name: proportion, dtype: float64

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


X = pd.read_csv("preprocessed_datasets/titanic_data.csv")
y = pd.read_csv("preprocessed_datasets/titanic_target.csv").squeeze()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
print(accuracy_score(model.predict(X_test), y_test))

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
})


feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df

0.8383458646616542


,Feature,Importance
1,Sex,0.428464
0,Pclass,0.142382
6,Ticket_number,0.114816
3,Fare,0.103789
5,Family_size,0.087623
2,Age,0.052393
4,Title,0.049686
8,Embarked_code,0.010685
7,Ticket_prefix_code,0.010160
